In [ ]:
# Block NULL: Readme

# Note by lzh: Always run Block A and Block B, and run Block C and / or Block D if necessary.
# Remeber to restart before conducting each experiment

In [ ]:
# Block A: Set up matplotlib and jupyter notebook

# Note by lzh: use %matplotlib inline to force plots to be static, use %matplotlib notebook
# to permit interactive plots
%matplotlib notebook

In [ ]:
# Block B: Import packages and definition of NN

import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from random import random
from math import sqrt

from math import sin, pi

import datetime

from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

class block(nn.Module):
    def __init__(self,inplanes,outplanes):
        super(block,self).__init__()
        self.fc = nn.Linear(inplanes,outplanes)

    def forward(self,x):
        x = F.relu(self.fc(x))
        return x

class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        layer = [block(2,50)]
        for i in range(10):
            layer.append(block(50,50))

        self.fc = nn.Linear(50,1)
        layer.append(self.fc)
        self.forward_prop=nn.Sequential(*layer)


    def forward(self, x):
        return self.forward_prop(x)

f = lambda x:sqrt(1-x**2)

test_func = lambda x:10*sin(2*pi*x)
n =  20
data_generate = [i/n for i in range(n)]

print(data_generate)
dataset = Variable(torch.Tensor(list(map(lambda x:[x,f(x)],data_generate))))
data_test = Variable(torch.Tensor(list(map(test_func,data_generate))))

Net = TestNet()
#print(data_test)
print(data_test)

learning_rate = 1e-4

criterian = nn.MSELoss()
optimizer = optim.Adam(Net.parameters(),lr=learning_rate)

min_los = float('inf')

In [ ]:
# Block C: Do regression with NN and save the model

model_filename = "Model-" + datetime.datetime.now().strftime("%Y%m%d%H%M%S%f") + ".mdlpkl"

print("Filename of the model is", model_filename)

iter_time = 50000
print_factor = 1000
for i in range(iter_time):
    output = Net(dataset)
    loss = criterian(output,data_test)
    loss.backward()
    optimizer.step()

    running_loss = 0.
    running_acc = 0.
    running_loss += loss.data[0]


    running_loss /= len(dataset)
    if running_loss < min_los:
        min_los = running_loss
        torch.save(Net, model_filename)

    if i % print_factor == 0:
        print("[%d/%d] Loss: %.5f" % (i + 1, iter_time, running_loss))

print(min_los)

In [ ]:
# Block D: Draw figures with matplotlib

# Note by lzh: If you want to load 2dmodel_.pkl, uncomment the following sentence
#     for 2dmodel_.pkl is generated from a different mechanism
# from test2D import  f,n,TestNet,test_func,block

testnet = torch.load('Model-20170707142537845436.pkl')
testnet.eval()

def map_func(func):
    return lambda x:list(map(lambda x:func([[x[0],x[1]]]),x))

n1=32
data_test = [[(i/n,j/n) for j in range(n1)]for i in range(n1)]
data_plot = list(map(lambda x:list(map(lambda y:max(testnet.forward(Variable(torch.Tensor([[y[0],y[1]]]))).data.numpy()[0]),x)),data_test))

fig = plt.figure()
ax = Axes3D(fig)
X = np.arange(0, 1, 1/n1)
Y = np.arange(0, 1, 1/n1)
X, Y = np.meshgrid(X, Y)

ax.plot_surface(X, Y, data_plot, rstride=1, cstride=1, cmap='rainbow')

for i in range(n):
    ax.scatter(f(i/n),i/n,test_func(i/n))
    ax.scatter( f(i / n),i / n, alpha=0.4,marker='+')

plt.show()